In [1]:
import patient as p
import tensorflow as tf
import numpy as np
import myLib
import consts as c

In [ ]:
tabSegments = []
for pStr in c.trainPatients:
    tabSegments = tabSegments + p.patient(pStr).getLabeledSegments()


x_train,y_train = myLib.processDF(tabSegments)
x_validation,y_validation = myLib.processDF(p.patient(c.validationPatient).getLabeledSegments())
x_test,y_test = myLib.processDF(p.patient(c.testPatient).getLabeledSegments())




Loading data of patient MSEL_00172


  6%|████▉                                                                            | 15/248 [00:00<00:10, 22.16it/s]


13120 seconds is too close to a positve label.
Skipping to next timestamp


 11%|█████████▏                                                                       | 28/248 [00:01<00:08, 25.25it/s]


27260 seconds is too close to a positve label.
Skipping to next timestamp


...

 40%|███████████████████████████████▋                                                | 160/404 [00:07<00:10, 22.92it/s]


159570 seconds is too close to a positve label.
Skipping to next timestamp


 51%|████████████████████████████████████████▌                                       | 205/404 [00:09<00:09, 21.57it/s]


203000 seconds is too close to a positve label.
Skipping to next timestamp


 57%|█████████████████████████████████████████████▎                                  | 229/404 [00:10<00:08, 20.02it/s]

In [ ]:
print(max(x_test[0][0]))

In [ ]:
print(f"train data: {len(x_train)}\t validation: {len(x_validation)}\t test:{len(x_test)}")
unique, counts = np.unique(y_train, return_counts=True)

print(f"TRAIN:     Negatives: {counts[0]} Positives: {counts[1]}")
unique, counts = np.unique(y_validation, return_counts=True)
print(f"VALIDATION Negatives: {counts[0]} Positves: {counts[1]}")
unique, counts = np.unique(y_test, return_counts=True)
print(f"TEST       Negatives: {counts[0]} Positves: {counts[1]}")

In [ ]:

countPos = 0
for y in y_train:
    if y == 1:
        countPos+=1
countNeg = len(y_train)-countPos

print(f"There are {countPos} positves and {countNeg} negatives")

classWeights = {0:countNeg,1:countPos}

In [ ]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(2,activation=tf.nn.softmax))

#model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[
          tf.keras.metrics.BinaryAccuracy(name='accuracy')
      ]
)

In [ ]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

#model.fit(x_train,y_train,epochs=20,class_weight=classWeights)

model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS)

In [ ]:
val_loss,val_acc = model.evaluate(x_test,y_test)
print(f"loss: {val_loss} acc: {val_acc}")

In [ ]:
predictions = model.predict(x_test)
res = [np.argmax(x) for x in predictions]
for i in range(len(y_test)):
    print(f"{y_test[i]} => {res[i]}")